In [ ]:
!pip install panns_inference transformers librosa soundfile --quiet

In [ ]:
import numpy as np
import json
import librosa
import os
import torch
from panns_inference import AudioTagging, labels

# 제외할 태그 & 목적 기반 태그 정의
EXCLUDE = [
    "Speech", "Male speech", "Female speech", "Narration", "Babbling",
    "Music", "Musical instrument", "Singing", "Choir",
    "Inside", "Reverberation", "Silence", "Buzz", "Hum"
]

TARGET_KEYWORDS = [
    "Rain", "Thunder", "Wind", "Water", "Drop", "Splash", "Stream", "Gush",
    "Footsteps", "Step", "Walk", "Running",
    "Door", "Knock", "Slam", "Open", "Close",
    "Car", "Vehicle", "Truck", "Motor", "Engine", "Traffic",
    "Fire", "Crackle"
]

def filter_environmental_tags(tag_list, score_list):
    filtered_tags = []
    filtered_scores = []

    for lbl, sc in zip(tag_list, score_list):
        # EXCLUDE 태그 제거
        if any(ex.lower() in lbl.lower() for ex in EXCLUDE):
            continue

        # 목적 기반 태그만 포함
        if any(key.lower() in lbl.lower() for key in TARGET_KEYWORDS):
            filtered_tags.append(lbl)
            filtered_scores.append(sc)

    return filtered_tags, filtered_scores

# PANNs 모델 로드
model = AudioTagging()

audio_dir = "/content/drive/MyDrive/TAVE 16th 심화플젝/data/split_scenes/clips_2_audio_wav"
audio_files = [f for f in os.listdir(audio_dir) if f.endswith(".wav")]

results = {}


for file in audio_files:
    file_path = os.path.join(audio_dir, file)

    # 오디오 로드 (PANNs 요구 sr=32k)
    audio, sr = librosa.load(file_path, sr=32000)
    audio = audio[np.newaxis, :]

    # 추론
    clipwise_output, embedding = model.inference(audio)
    clipwise_output = clipwise_output[0]

    # Top-K 추출
    top_k = 5
    top_indices = clipwise_output.argsort()[-top_k:][::-1]

    raw_labels = [labels[idx] for idx in top_indices]
    raw_scores = [float(clipwise_output[idx]) for idx in top_indices]

    # 필터링 적용
    filtered_labels, filtered_scores = filter_environmental_tags(raw_labels, raw_scores)

    # 결과 저장
    results[file] = {
        "raw_tags": raw_labels,
        "raw_scores": raw_scores,
        "filtered_tags": filtered_labels,
        "filtered_scores": filtered_scores
    }

    print(f"\n🎬 {file}")
    print("  원본 Top-5 :", raw_labels)
    print("  필터링 후 :", filtered_labels)


save_path = "/content/drive/MyDrive/TAVE 16th 심화플젝/data/background_audio/background_labels_2.json"

with open(save_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print("\n저장 완료:", save_path)